In [ ]:
#| default_exp merging
import sys
from pathlib import Path
# Insert in Path Project Directory
sys.path.insert(0, str(Path().cwd().parent))
%load_ext autoreload
%autoreload 2

# Mesclagem
> Funções auxiliares para mesclar e limpar as várias fontes de dados

In [ ]:
#| export
from decimal import Decimal
from itertools import product
import pandas as pd
from geopy.distance import geodesic

from anateldb.constants import ENTIDADES
from anateldb.format import input_coordenates, scrape_dataframe, df_optimize
from anateldb.reading import read_icao, read_aisw, read_aisg, read_aero, read_base
MAX_DIST = 0.5 #Km
COLS = ['Frequency', 'Latitude', 'Longitude', 'Description']

### Aeronáutica
Funções auxiliares para mesclar registros que são iguais das diversas bases da aeronáutica, i.e. estão a uma distância menor que `DIST` e verificar a validade da mesclagem

In [ ]:
#| export
def check_add_row(df, # DataFrame para mesclar adicionar o registro
                  f, # Frequência (MHz) em análise do registro 
                  rows, # Lista de registros para mesclar
                  dicts, # Dicionário fonte dos registros
):
    """Mescla os registros em `rows` de frequência `f` e os adiciona como uma linha do DataFrame `df`
    Os registros em `rows` somente são mesclados se ainda constarem nos dicionários fonte `dicts`
    Após a mesclagem, os registros são removidos dos dicionários fonte `dicts`   
    """
    if all(row.Index in dict for row, dict in zip(rows, dicts)):
        lat = sum(row.Latitude for row in rows) / len(rows)
        long = sum(row.Longitude for row in rows) / len(rows)
        desc = ' | '.join(row.Description for row in rows)
        d = {'Frequency': f, 'Latitude': lat, 'Longitude': long, 'Description': desc}
        for row, dict in zip(rows, dicts):
            dict.pop(row.Index)
        return pd.concat([df, pd.DataFrame(d, index=[0])], ignore_index=True)
    return df


In [ ]:
#| export
def get_subsets(f, # Frequência (MHz) em análise do registro
                *dfs, # Conjunto de DataFrames a serem analisados
):
    """Retorna os subconjuntos de registros de frequência `f` em cada dataframe `dfs`
    Os subconjuntos são retornados em forma de dicionário, onde a chave é o índice do registro
    """
    return [{s.Index: s for s in df.loc[df.Frequency == f, COLS].itertuples()} for df in dfs]


In [ ]:
#| export
def merge_closer(frequencies, # Lista de frequências em comum
                 df, # DataFrame de saída
                 df_left, # DataFrame 1 de entrada da esquerda
                 df_right # DataFrame 2 de entrada da direita
):
    """Mescla os registros de frequência `frequencies` de `df_left` e `df_right` em `df`
    Essa função é utilizada para mesclar registros que possuem frequências em comum listadas em `frequencies`
    Os registros são mesclados se a distância entre eles for menor que `MAX_DIST`
    do contrário são adicionados individualmente como uma linha no DataFrame de saída `df`	
    """
    for f in frequencies:
        sa, sb = get_subsets(f, df_left, df_right)
        if all([sa, sb]): # Somente há registros para mesclar se estiverem nos dois conjuntos
            for fa, fb in list(product(sa.copy().values(), sb.copy().values())):
                if geodesic((fa.Latitude, fa.Longitude), (fb.Latitude, fb.Longitude)).km <= MAX_DIST:
                    df = check_add_row(df, f, [fa, fb], [sa, sb]) 
        for reg in [sa, sb]: # Do contrário os registros são adicionados individualmente ao DataFrame
            for r in reg.copy().values():
                df = check_add_row(df, f, [r], [reg])
    return df


In [ ]:
#| export
def merge_single(frequencies, # Lista de frequências em comum
                 df, # DataFrame de saída
                 df_left # DataFrame de entrada
):
    """Mescla os registros de frequência `frequencies` de `df_left` em `df`"""
    for f in frequencies:
        if sa := get_subsets(f, df_left)[0]:
            for fa in sa.copy().values():
                df = check_add_row(df, f, [fa], [sa])
    return df

In [ ]:
#| export
def merge_triple(frequencies, # Lista de frequências em comum
                 df, # DataFrame de saída 
                 df_left, # DataFrame 1 de entrada
                 df_middle, # DataFrame 2 de entrada 
                 df_right, # DataFrame 3 de entrada 
):
    """Mescla os registros de frequência `frequencies` de `df_left`, `df_middle` e `df_right` em `df`
    Essa função é utilizada para mesclar registros que possuem frequências em comum listadas em `frequencies`
    Os registros são mesclados se a distância entre eles for menor que `MAX_DIST`
    do contrário são adicionados individualmente como uma linha no DataFrame de saída `df`
    """
    for f in frequencies:
        sa, sb, sc = get_subsets(f, df_left, df_middle, df_right)
        if all([sa, sb, sc]):
            for fa, fb, fc in list(product(sa.copy().values(), sb.copy().values(), sc.copy().values())):
                dab = geodesic((fa.Latitude, fa.Longitude), (fb.Latitude, fb.Longitude)).km
                dac = geodesic((fa.Latitude, fa.Longitude), (fc.Latitude, fc.Longitude)).km
                dbc = geodesic((fb.Latitude, fb.Longitude), (fc.Latitude, fc.Longitude)).km
                if all(d <= MAX_DIST  for d in [dab, dac, dbc]):
                    df = check_add_row(df, f, [fa, fb, fc], [sa, sb, sc])
                elif all(d > MAX_DIST  for d in [dac, dbc]):
                    df = check_add_row(df, f, [fa, fb], [sa, sb])
                elif all(d > MAX_DIST  for d in [dab, dac]):
                    df = check_add_row(df, f, [fa, fc], [sb, sc])
                elif all(d > MAX_DIST for d in [dab, dbc]):
                    df = check_add_row(df, f, [fa, fc], [sa, sc])
        for reg in [sa, sb, sc]:
            for r in reg.copy().values():
                df = check_add_row(df, f, [r], [reg])        
    return df

In [ ]:
#| export
def check_merging(df, # DataFrame de saída
                  icao, # DataFrame fonte 1
                  aisw, # DataFrame fonte 2 
                  aisg, # DataFrame fonte 3
):
    """Verifica a validade da mesclagem dos registros de `icao`, `aisw` e `aisg` em `df`"""
    three_merges = df[df.Description.str.contains('\|.*\|')]
    two_merges = df[(df.Description.str.contains('[\|]{1}')) & (~df.index.isin(three_merges.index))]
    no_merge = df[~df.Description.str.contains('[\|]{1}')]
    return len(no_merge) + len(two_merges) * 2 + len(three_merges) * 3 == len(icao) + len(aisw) + len(aisg)


In [ ]:
#| export
def get_frequencies_set(df1, df2, df3):
    """Retorna todos os conjuntos de frequências do Diagrama de Venn entre os registros de `df1`, `df2` e `df3`"""
    f1 = set(df1.Frequency.tolist())
    f2 = set(df2.Frequency.tolist())
    f3 = set(df3.Frequency.tolist())
    ABC = f1.intersection(f2).intersection(f3)
    AB = f1.intersection(f2).difference(ABC)
    BC = f2.intersection(f3).difference(ABC)
    AC = f1.intersection(f3).difference(ABC)
    A = f1.difference(ABC).difference(AB).difference(AC)
    B = f2.difference(ABC).difference(AB).difference(BC)
    C = f3.difference(ABC).difference(BC).difference(AC)
    return A, B, C, AB, AC, BC, ABC

In [ ]:
#| export
def merge_aero(folder, # Pasta onde estão os arquivos de entrada
):
    """Mescla os registros de mesma frequência e próximos dos arquivos da aeronáutica em `folder`"""
    icao = read_icao(folder).drop(columns=['Service', 'Station'])
    aisw = read_aisw(folder).drop(columns=['Service', 'Station'])
    aisg = read_aisg(folder).drop(columns=['Service', 'Station'])
    df = pd.DataFrame(columns=['Frequency', 'Latitude', 'Longitude', 'Description'])
    A, B, C, AB, AC, BC, ABC = get_frequencies_set(icao, aisw, aisg)
    df = merge_closer(AB, df, icao, aisw)
    df = merge_closer(AC, df, icao, aisg)
    df = merge_closer(BC, df, aisw, aisg)
    df = merge_single(A, df, icao)
    df = merge_single(B, df, aisw)
    df = merge_single(C, df, aisg)
    df = merge_triple(ABC, df, icao, aisw, aisg)
    if not check_merging(df, icao, aisw, aisg):
        raise ValueError("Divergência na contagem de linhas entre as bases individuais e a combinação")
    return df


In [ ]:
#| echo : False
def _merge_dfs(df1, df2, on, how="left"):
    """Merge two dataframes with the same columns and records"""
    df = pd.merge(df1, df2, on=on, how=how)
    x = df.Description_x.notna()
    y = df.Description_y.notna()
    df.loc[x & y, "Description"] = (
        df.loc[x & y, "Description_x"] + " | " + df.loc[x & y, "Description_y"]
    )
    df.loc[x & ~y, "Description"] = df.loc[x & ~y, "Description_x"]
    df.loc[~x & y, "Description"] = df.loc[~x & y, "Description_y"]
    df.loc[x & y, "Latitude"] = (
        df.loc[x & y, "Latitude_x"] + df.loc[x & y, "Latitude_y"]
    ) / 2
    df.loc[x & y, "Longitude"] = (
        df.loc[x & y, "Longitude_x"] + df.loc[x & y, "Longitude_y"]
    ) / 2
    df.loc[x & ~y, "Latitude"] = df.loc[x & ~y, "Latitude_x"]
    df.loc[x & ~y, "Longitude"] = df.loc[x & ~y, "Longitude_x"]
    df.loc[~x & y, "Latitude"] = df.loc[~x & y, "Latitude_y"]
    df.loc[~x & y, "Longitude"] = df.loc[~x & y, "Longitude_y"]
    if "Service_x" in df.columns and "Service_y" in df.columns:
        df.loc[x, "Service"] = df.loc[x, "Service_x"]
        df.loc[~x & y, "Service"] = df.loc[~x & y, "Service_y"]
    return df.loc[:, [c for c in df.columns if "_" not in c]]


def aero_common(dfa, dfb, dfc):
    cols = ["Frequency", "Station"]
    a = dfa[dfa.Station != -1].reset_index(drop=True)
    b = dfb[dfb.Station != -1].reset_index(drop=True)
    c = dfc[dfc.Station != -1].reset_index(drop=True)
    common = _merge_dfs(a, b, cols, how="outer")
    common = _merge_dfs(common, c, cols, how="outer")
    common.drop_duplicates(inplace=True, keep="first")
    return common.reset_index(drop=True)

In [ ]:
#| echo : False
def aero_new(dfa, dfb, dfc, dist=500):
    cols = ["Frequency"]
    a = (
        dfa[dfa.Station == -1]
        .drop(columns=["Service", "Station"])
        .reset_index(drop=True)
    )
    b = (
        dfb[dfb.Station == -1]
        .drop(columns=["Service", "Station"])
        .reset_index(drop=True)
    )
    c = (
        dfc[dfc.Station == -1]
        .drop(columns=["Service", "Station"])
        .reset_index(drop=True)
    )
    abc = (
        pd.merge(a, b, on=cols, how="outer")
        .merge(c, on=cols, how="outer")
        .reset_index(drop=True)
        .drop_duplicates(keep="first")
    )
    x = abc.Description_x.notna()
    y = abc.Description_y.notna()
    z = abc.Description.notna()
    new = pd.DataFrame(columns=["Frequency", "Latitude", "Longitude", "Description"])

    for c, n in zip([(x & ~y & ~z), (~x & y & ~z), (~x & ~y & z)], ["_x", "_y", ""]):
        temp = abc.loc[
            c, ["Frequency", f"Latitude{n}", f"Longitude{n}", f"Description{n}"]
        ]
        temp.columns = ["Frequency", "Latitude", "Longitude", "Description"]
        new = pd.concat([new, temp], ignore_index=True)

    new = new.drop_duplicates(keep="first").reset_index(drop=True)

    for c, (left, right) in zip(
        [(x & y & ~z), (x & ~y & z), (~x & y & z)],
        [("_x", "_y"), ("_x", ""), ("_y", "")],
    ):
        for row in abc[c].itertuples():
            d = geodesic(
                (getattr(row, f"Latitude{left}"), getattr(row, f"Longitude{left}")),
                (getattr(row, f"Latitude{right}"), getattr(row, f"Longitude{right}")),
            ).m
            if d < dist:
                f = row.Frequency
                lat = (
                    getattr(row, f"Latitude{left}") + getattr(row, f"Latitude{right}")
                ) / 2
                lon = (
                    getattr(row, f"Longitude{left}") + getattr(row, f"Longitude{right}")
                ) / 2
                d = (
                    getattr(row, f"Description{left}")
                    + " | "
                    + getattr(row, f"Description{right}")
                )
                new = pd.concat(
                    [
                        new,
                        pd.DataFrame(
                            [[f, lat, lon, d]],
                            columns=[
                                "Frequency",
                                "Latitude",
                                "Longitude",
                                "Description",
                            ],
                        ),
                    ],
                    ignore_index=True,
                )
            else:
                l = (
                    abc.loc[
                        row.Index,
                        [
                            "Frequency",
                            f"Latitude{left}",
                            f"Longitude{left}",
                            f"Description{left}",
                        ],
                    ]
                    .to_frame()
                    .T
                )
                l.columns = new.columns
                r = (
                    abc.loc[
                        row.Index,
                        [
                            "Frequency",
                            f"Latitude{right}",
                            f"Longitude{right}",
                            f"Description{right}",
                        ],
                    ]
                    .to_frame()
                    .T
                )
                r.columns = new.columns
                new = pd.concat([new, l, r], ignore_index=True)

    new = new.drop_duplicates(keep="first").reset_index(drop=True)

    for row in abc[x & y & z].itertuples():
        d1 = geodesic(
            (getattr(row, "Latitude_x"), getattr(row, "Longitude_x")),
            (getattr(row, "Latitude_y"), getattr(row, "Longitude_y")),
        ).m
        d2 = geodesic(
            (getattr(row, "Latitude_x"), getattr(row, "Longitude_x")),
            (getattr(row, "Latitude"), getattr(row, "Longitude")),
        ).m
        d3 = geodesic(
            (getattr(row, "Latitude_y"), getattr(row, "Longitude_y")),
            (getattr(row, "Latitude"), getattr(row, "Longitude")),
        ).m
        if d1 < dist and d2 < dist and d3 < dist:
            f = row.Frequency
            lat = (row.Latitude_x + row.Latitude_y + row.Latitude) / 3
            lon = (row.Longitude_x + row.Longitude_y + row.Longitude) / 3
            d = " | ".join([row.Description_x, row.Description_y, row.Description])
            new = pd.concat(
                [
                    new,
                    pd.DataFrame(
                        [[f, lat, lon, d]],
                        columns=["Frequency", "Latitude", "Longitude", "Description"],
                    ),
                ],
                ignore_index=True,
            )
        elif d1 < dist and d2 < dist:
            f = row.Frequency
            lat = (row.Latitude_x + row.Latitude_y) / 2
            lon = (row.Longitude_x + row.Longitude_y) / 2
            d = " | ".join([row.Description_x, row.Description_y])
            new = pd.concat(
                [
                    new,
                    pd.DataFrame(
                        [[f, lat, lon, d]],
                        columns=["Frequency", "Latitude", "Longitude", "Description"],
                    ),
                ],
                ignore_index=True,
            )
        elif d1 < dist and d3 < dist:
            f = row.Frequency
            lat = (row.Latitude_x + row.Latitude) / 2
            lon = (row.Longitude_x + row.Longitude) / 2
            d = " | ".join([row.Description_x, row.Description])
            new = pd.concat(
                [
                    new,
                    pd.DataFrame(
                        [[f, lat, lon, d]],
                        columns=["Frequency", "Latitude", "Longitude", "Description"],
                    ),
                ],
                ignore_index=True,
            )
        elif d2 < dist and d3 < dist:
            f = row.Frequency
            lat = (row.Latitude_y + row.Latitude) / 2
            lon = (row.Longitude_y + row.Longitude) / 2
            d = " | ".join([row.Description_y, row.Description])
            new = pd.concat(
                [
                    new,
                    pd.DataFrame(
                        [[f, lat, lon, d]],
                        columns=["Frequency", "Latitude", "Longitude", "Description"],
                    ),
                ],
                ignore_index=True,
            )
        else:
            l = (
                abc.loc[
                    row.Index,
                    ["Frequency", "Latitude_x", "Longitude_x", "Description_x"],
                ]
                .to_frame()
                .T
            )
            l.columns = new.columns
            r = (
                abc.loc[
                    row.Index,
                    ["Frequency", "Latitude_y", "Longitude_y", "Description_y"],
                ]
                .to_frame()
                .T
            )
            r.columns = new.columns
            s = (
                abc.loc[
                    row.Index, ["Frequency", "Latitude", "Longitude", "Description"]
                ]
                .to_frame()
                .T
            )
            s.columns = new.columns
            new = pd.concat([new, l, r, s], ignore_index=True)

    new = new.drop_duplicates(keep="first").reset_index(drop=True)
    new["Service"] = pd.NA
    new["Station"] = pd.NA
    return new

In [ ]:
#| echo : False

def merge_aero(df, common, new):
    """Mescla a base da Anatel com as tabelas retiradas da Aeronáutica"""
    common = common.loc[:, ["Frequency", "Description", "Service", "Station"]]
    df["Description"] = df.Description.astype("string")
    df = pd.merge(df, common, on=["Frequency", "Service", "Station"], how="left")
    df.loc[df.Description_y.notna(), "Description_x"] = (
        df.loc[df.Description_y.notna(), "Description_x"]
        + " | "
        + df.loc[df.Description_y.notna(), "Description_y"]
    )
    df.drop(columns=["Description_y"], inplace=True)
    df.rename(columns={"Description_x": "Description"}, inplace=True)
    new['Class'] = pd.NA
    new['BW'] = pd.NA
    return (
        pd.concat([df, new], ignore_index=True)
        .sort_values("Frequency")
        .reset_index(drop=True)
    )